# Usage

Just import the `registers` package and use one of functions:
* `read(file, **fmtparams)` - parses CSV `file`; `fmtparams` are passed to the csv `DictReader`

In [8]:
import sys
import csv
from pprint import pp as pprint

import registers

## Exmple usage

In [21]:
from enum import Enum
from IPython.display import display, Markdown

with open('sofar.csv') as f:
    regs = registers.read(f, delimiter=';')
    md = """
|Addr|Name|Type|Unit|Desc|Groups|
|----|----|----|----|----|------|""".splitlines()
          
    for r in regs:
          md.append(f'|{r.address}|{r.name}|{r.type.name}|{r.unit.name if type(r.unit) is Enum else r.unit}|{r.description}|{r.groups}|')
          
    display(Markdown('\n'.join(md)))


|Addr|Name|Type|Unit|Desc|Groups|
|----|----|----|----|----|------|
|1028|SysState|U16|<enum 'SysState'>|System state|{'status', 'main'}|
|1048|Temperature_Env1|I16|degC|System temperature|{'main', 'status', 'temp'}|
|1049|Temperature_Env2|I16|degC|System temperature 2|{'status', 'temp'}|
|1029|Fault1|U16||Fault1|{'fault', 'status'}|
|1030|Fault2|U16||Fault2|{'fault', 'status'}|
|1031|Fault3|U16||Fault3|{'fault', 'status'}|
|5248|EvLst_ID1|U16||EvLst_ID1|{'status', 'event'}|
|5249|EvLst_YM1|U16||EvLst_YM1|{'status', 'event'}|
|5250|EvLst_DH1|U16||EvLst_DH1|{'status', 'event'}|
|5251|EvLst_MS1|U16||EvLst_MS1|{'status', 'event'}|
|5252|EvLst_ID2|U16||EvLst_ID2|{'status', 'event'}|
|5253|EvLst_YM2|U16||EvLst_YM2|{'status', 'event'}|
|5254|EvLst_DH2|U16||EvLst_DH2|{'status', 'event'}|
|5255|EvLst_MS2|U16||EvLst_MS2|{'status', 'event'}|
|1156|Frequency_Grid|U16|Hz|Grid frequency|{'metrics', 'grid', 'main'}|
|1157|ActivePower_Output_Total|I16|kW|ActivePower_Output_Total|{'power', 'metrics', 'grid', 'main'}|
|1159|ApparentPower_Output_Total|I16|kW|ApparentPower_Output_Total|{'power', 'metrics', 'grid', 'apparent'}|
|1160|ActivePower_PCC_Total|I16|kW|ActivePower_PCC_Total|{'power', 'metrics', 'pcc', 'main'}|
|1162|ApparentPower_PCC_Total|I16|kW|ApparentPower_PCC_Total|{'power', 'metrics', 'pcc', 'apparent'}|
|1284|ActivePower_Load_Total|I16|kW|ActivePower_Load_Total|{'power', 'metrics', 'load', 'main'}|
|1286|ApparentPower_Load_Total|I16|kW|ApparentPower_Load_Total|{'power', 'metrics', 'load', 'apparent'}|
|1165|Voltage_Phase_R|U16|V|Grid phase R voltage|{'line', 'metrics', 'voltage'}|
|1166|Current_Output_R|U16|A|Current_Output_R|{'line', 'metrics', 'current'}|
|1176|Voltage_Phase_S|U16|V|Grid phase S voltage|{'line', 'metrics', 'voltage'}|
|1177|Current_Output_S|U16|A|Current_Output_S|{'line', 'metrics', 'current'}|
|1187|Voltage_Phase_T|U16|V|Grid phase T voltage|{'line', 'metrics', 'voltage'}|
|1188|Current_Output_T|U16|A|Current_Output_T|{'line', 'metrics', 'current'}|
|1198|ActivePower_PV_Ext|U16|kW|ActivePower_PV_Ext|{'pv', 'metrics', 'power'}|
|1199|ActivePower_Load_Sys|U16|kW|ActivePower_Load_Sys|{'power', 'metrics', 'sys', 'load'}|
|1412|Voltage_PV1|U16|V|PV MPPT 1 voltage|{'pv', 'metrics', 'voltage'}|
|1413|Current_PV1|U16|A|PV MPPT 1 current|{'pv', 'metrics', 'current'}|
|1414|Power_PV1|U16|kW|PV MPPT 1 power|{'pv', 'metrics', 'main', 'power'}|
|1415|Voltage_PV2|U16|V|PV MPPT 2 voltage|{'pv', 'metrics', 'voltage'}|
|1416|Current_PV2|U16|A|PV MPPT 2 current|{'pv', 'metrics', 'current'}|
|1417|Power_PV2|U16|kW|PV MPPT 2 power|{'pv', 'metrics', 'main', 'power'}|
|1540|Voltage_Bat1|U16|V|Battery 1 voltage|{'metrics', 'bat', 'voltage'}|
|1541|Current_Bat1|I16|A|Battery 1 current|{'current', 'metrics', 'bat'}|
|1542|Power_Bat1|I16|kW|Battery 1 power|{'power', 'metrics', 'bat'}|
|1546|ChargeCycle_Bat1|U16|cycle|Battery 1 cycles|{'metrics', 'cycle', 'bat'}|
|1547|Voltage_Bat2|U16|V|Battery 2 voltage|{'metrics', 'bat', 'voltage'}|
|1548|Current_Bat2|I16|A|Battery 2 current|{'current', 'metrics', 'bat'}|
|1549|Power_Bat2|I16|kW|Battery 2 power|{'power', 'metrics', 'bat'}|
|1553|ChargeCycle_Bat2|U16|cycle|Battery 2 cycles|{'metrics', 'cycle', 'bat'}|
|1924|ArcStrength_Chn1|I16||ArcStrength_Chn1|{'metrics', 'arc'}|
|1925|ArcStrength_Chn2|I16||ArcStrength_Chn2|{'metrics', 'arc'}|
|8196|Country_Code|U16||Safety settings - Country code|{'info'}|
|8199|Safety_Name|ASCII||Safety settings - name|{'info'}|
|2112|VoltageConfig|BITMAP|<enum 'VoltageConfig'>|Used grid protection parameters|{'ovp', 'info'}|
|2113|RatedVoltage|U16|V|RatedVoltage|{'info'}|
|2114|FirstOvervoltageProtectionValue|U16|V|FirstOvervoltageProtectionValue|{'ovp', 'info'}|
|2115|FirstOvervoltageProtectionTime|U16|10ms|FirstOvervoltageProtectionTime|{'ovp', 'info'}|
|2116|SecondOvervoltageProtectionValue|U16|V|SecondOvervoltageProtectionValue|{'ovp', 'info'}|
|2117|SecondOvervoltageProtectionTime|U16|10ms|SecondOvervoltageProtectionTime|{'ovp', 'info'}|
|2118|ThirdOvervoltageProtectionValue|U16|V|ThirdOvervoltageProtectionValue|{'ovp', 'info'}|
|2119|ThirdOvervoltageProtectionTime|U16|10ms|ThirdOvervoltageProtectionTime|{'ovp', 'info'}|
|2120|FirstUnderVoltageProtectionValue|U16|V|FirstUnderVoltageProtectionValue|{'ovp', 'info'}|
|2121|FirstUndervoltageProtectionTime|U16|10ms|FirstUndervoltageProtectionTime|{'ovp', 'info'}|
|2122|SecondUnderVoltageProtectionValue|U16|V|SecondUnderVoltageProtectionValue|{'info', 'uvp'}|
|2123|SecondUndervoltageProtectionTime|U16|10ms|SecondUndervoltageProtectionTime|{'info', 'uvp'}|
|2124|ThirdUnderVoltageProtectionValue|U16|V|ThirdUnderVoltageProtectionValue|{'info', 'uvp'}|
|2125|ThirdUndervoltageProtectionTime|U16|10ms|ThirdUndervoltageProtectionTime|{'info', 'uvp'}|
|2126|10MinOvervoltageProtectionValue|U16|V|10MinOvervoltageProtectionValue|{'ovp', 'info'}|
|2304|RemoteConfig|BITMAP|<enum 'RemoteConfig'>|RemoteConfig|{'info'}|
|2305|ActiveOutputLimit|U16|%|ActiveOutputLimit|{'info'}|
|2306|ActiveOutputDownSpeed|U16|%Pn/min|ActiveOutputDownSpeed|{'info'}|
|2307|GridVoltageDropStart|U16|V|GridVoltageDropStart|{'info'}|
|2308|GridVoltageDropStop|U16|V|GridVoltageDropStop|{'info'}|
|2309|GridVoltageDropMinPower|I16|%|GridVoltageDropMinPower|{'info'}|
|2310|OvervoltageDownSpeed|U16|%Pn/min|OvervoltageDownSpeed|{'info'}|
|2322|RefluxPower|U16|%|RefluxPower|{'info'}|
|2323|RefluxOVloadTime|U16|10ms|RefluxOVloadTime|{'info'}|
|4131|AntiReflux_Control|U16|<enum 'AntiReflux_Control'>|AntiReflux_Control|{'status'}|
|4132|AntiReflux_Power|U16|10W|AntiReflux_Power|{'status'}|
|4152|UnbalancedSupport_Control|U16|<enum 'UnbalancedSupport_Control'>|UnbalancedSupport_Control|{'status'}|

## Units may be Enums and IntFlags

In [29]:
with open('sofar.csv') as f:
    regs = registers.read(f, delimiter=';')
    md = []
    for r in regs:
        if type(r.unit) is not str:
            md.append(f'* {r.name}: {r.type.name}')
            md += [f'  * {r.unit[n].value}: {n}' for n in r.unit.__members__]
    display(Markdown('\n'.join(md)))

* SysState: U16
  * 0: Waiting
  * 1: Checking
  * 2: On-grid
  * 3: Emergency Mode
  * 4: Recoverable Fault
  * 5: Permanent Fault
  * 6: Upgarding
  * 7: Self-charging
* VoltageConfig: BITMAP
  * 64: OVP - 10 min.
  * 32: UVP 3
  * 16: UVP 2
  * 8: UVP 1
  * 4: OVP 3
  * 2: OVP 2
  * 1: OVP 1
* RemoteConfig: BITMAP
  * 1: AC power derate - active
  * 2: Remote on/off capability
  * 4: AC power derate - high volt.
  * 8: Batt. charging power derate - high volt.
  * 16: Batt. charging power derate - low volt.
  * 32: DRM0
  * 64: DRM1-8
  * 128: Reflux power overload
* AntiReflux_Control: U16
  * 0: disabled
  * 1: enabled
  * 2: enabled - avg. power mode
* UnbalancedSupport_Control: U16
  * 0: disabled
  * 1: enabled